# QuickBooks Commerce - Sales Forecasting Data Exploration

This notebook explores the synthetic sales data used for training the ensemble model (XGBoost + Holt-Winters).

**Model**: Ensemble v2.0.0 — XGBoost (60%) + Holt-Winters (40%)  
**Features**: 17 engineered features  
**Holdout R²**: 0.82

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Load Data

We use synthetic data with category-specific baselines and realistic seasonality.
The same data generation approach is used in `scripts/train_model.py`.

In production, you would load from:
- QuickBooks Commerce API (real merchant sales)
- FRED API (economic indicators)
- Yahoo Finance (market trends)

In [ ]:
# Generate synthetic sales data (matches data_service.py approach)
np.random.seed(42)  # Reproducible

def generate_sales_data(days=366):
    """Generate realistic synthetic sales data with category-specific baselines"""
    categories = [
        'Electronics', 'Clothing', 'Home & Garden',
        'Sports & Outdoors', 'Books & Media', 'Food & Beverages',
        'Health & Beauty', 'Toys & Games'
    ]
    
    # Category-specific base sales (matches trained model)
    category_base = {
        'Electronics': 1800,
        'Clothing': 1200,
        'Home & Garden': 900,
        'Sports & Outdoors': 1000,
        'Books & Media': 600,
        'Food & Beverages': 1500,
        'Health & Beauty': 800,
        'Toys & Games': 700,
    }
    
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)
    
    data = []
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    for date in date_range:
        for category in categories:
            base_sales = category_base.get(category, 1000) + np.random.uniform(-200, 200)
            
            # Monthly seasonality
            month_factor = 1.0 + 0.15 * np.sin(2 * np.pi * date.month / 12)
            
            # Weekend boost
            weekend_boost = 1.15 if date.weekday() >= 5 else 1.0
            
            # Random noise
            noise = np.random.normal(0, base_sales * 0.08)
            
            sales = max(0, base_sales * month_factor * weekend_boost + noise)
            
            data.append({
                'date': date,
                'category': category,
                'sales_units': int(sales),
                'revenue': round(sales * np.random.uniform(30, 80), 2)
            })
    
    return pd.DataFrame(data)

df = generate_sales_data()
print(f"Dataset shape: {df.shape}")
print(f"Categories: {df['category'].nunique()}")
print(f"Date range: {df['date'].min().date()} to {df['date'].max().date()}")
df.head()

## 2. Exploratory Data Analysis

In [ ]:
# Basic statistics
print("\nDataset Info:")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Categories: {df['category'].nunique()}")
print(f"Total records: {len(df)}")

print("\nSales Statistics by Category:")
df.groupby('category').agg({
    'sales_units': ['mean', 'std', 'min', 'max'],
    'revenue': 'sum'
}).round(2)

In [ ]:
# Sales trend over time
plt.figure(figsize=(14, 6))
for category in df['category'].unique():
    cat_data = df[df['category'] == category].groupby('date')['sales_units'].sum()
    plt.plot(cat_data.index, cat_data.values, label=category, alpha=0.7)

plt.title('Sales Trend by Category Over Time', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Sales Units')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
# Category performance comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Total sales by category
category_sales = df.groupby('category')['sales_units'].sum().sort_values(ascending=True)
category_sales.plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_title('Total Sales by Category', fontweight='bold')
axes[0].set_xlabel('Sales Units')

# Revenue by category
category_revenue = df.groupby('category')['revenue'].sum().sort_values(ascending=True)
category_revenue.plot(kind='barh', ax=axes[1], color='coral')
axes[1].set_title('Total Revenue by Category', fontweight='bold')
axes[1].set_xlabel('Revenue ($)')

plt.tight_layout()
plt.show()

In [ ]:
# Seasonality analysis
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Monthly pattern
monthly_sales = df.groupby('month')['sales_units'].mean()
axes[0].bar(monthly_sales.index, monthly_sales.values, color='teal')
axes[0].set_title('Average Daily Sales by Month', fontweight='bold')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Average Sales Units')
axes[0].set_xticks(range(1, 13))

# Weekly pattern
weekly_sales = df.groupby('day_of_week')['sales_units'].mean()
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
axes[1].bar(range(7), weekly_sales.values, color='purple')
axes[1].set_title('Average Sales by Day of Week', fontweight='bold')
axes[1].set_xlabel('Day of Week')
axes[1].set_ylabel('Average Sales Units')
axes[1].set_xticks(range(7))
axes[1].set_xticklabels(days)

plt.tight_layout()
plt.show()

## 3. Feature Engineering

In [ ]:
def create_features(df):
    """Create time-series features"""
    df = df.copy()
    
    # Time features
    df['year'] = df['date'].dt.year
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['day_of_week'] = df['date'].dt.dayofweek
    df['week_of_year'] = df['date'].dt.isocalendar().week
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    
    # Lag features
    for lag in [7, 14, 30]:
        df[f'sales_lag_{lag}'] = df.groupby('category')['sales_units'].shift(lag)
    
    # Rolling features
    for window in [7, 30]:
        df[f'sales_rolling_mean_{window}'] = df.groupby('category')['sales_units'].transform(
            lambda x: x.rolling(window=window, min_periods=1).mean()
        )
        df[f'sales_rolling_std_{window}'] = df.groupby('category')['sales_units'].transform(
            lambda x: x.rolling(window=window, min_periods=1).std()
        )
    
    return df

df_features = create_features(df)
print("\nFeatures created:")
print(df_features.columns.tolist())
df_features.head()

## 4. Save Processed Data

In [ ]:
# Save for model training
import os
os.makedirs('../data/processed', exist_ok=True)

df_features.to_csv('../data/processed/sales_features.csv', index=False)
print("Data saved to: ../data/processed/sales_features.csv")

## 5. Key Insights

1. **Category-Specific Baselines**: Electronics (~1800) is highest, Books & Media (~600) is lowest
2. **Weekend Effect**: ~15% sales boost on weekends
3. **Monthly Seasonality**: Cyclical pattern driven by sin(month/12)
4. **Noise**: ~8% standard deviation gives realistic variation

## Trained Model (v2.0.0)

The ensemble model uses these patterns:
- **XGBoost** (60% weight): 17 features including lag, rolling stats, time features
- **Holt-Winters** (40% weight): Per-category weekly seasonality (period=7)
- **Holdout R²**: 0.82 on last 30 days

To train the model:
```bash
cd backend
python -m scripts.train_model
```

Next steps for improvement:
- Integrate real sales data (QuickBooks API)
- Add external indicators (FRED API for GDP, inflation)
- Cross-validation for hyperparameter tuning
- Feature selection to reduce overfitting gap